In [3]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential

In [4]:
df = pd.read_csv('spam.csv')
df.head()

,label,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
df['target'] = df['label'].map( {'spam':1, 'ham':0 })

In [6]:
## b) Trainings- und Testdaten separieren
df_train = df.sample(frac=.8, random_state=11)
df_test = df.drop(df_train.index)
print(df_train.shape, df_test.shape)

(4458, 3) (1114, 3)


In [7]:
y_train = df_train['target'].values
y_test = df_test['target'].values
y_test.shape

(1114,)

In [8]:
X_train = df_train['sms'].values
X_test = df_test['sms'].values

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
word_dict = tokenizer.index_word

In [10]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
print(X_train_seq[:5])
print(df_train.iloc[0,:])
for el in X_train_seq[0]:
    print(word_dict[el], end=' ')

[[172, 211, 12, 13, 87, 92, 45, 8, 32, 3799, 231, 9, 7, 86, 6, 81, 1020, 5, 3800, 7, 1999, 11, 635, 241, 21, 25, 436, 928, 1110, 178, 131, 206, 929, 2564, 23, 1, 154, 80, 2, 110, 82, 48, 2, 135, 11, 929, 227, 98, 1639], [257, 307, 2, 1426, 2565, 6, 33, 30, 1245, 1246, 15, 49, 5, 337, 709, 7, 1427, 1428, 581, 68, 34, 2000, 88, 2, 2001], [22, 636, 13, 283, 211, 7, 26, 3, 17, 94, 1429, 67], [13, 296, 2, 30, 18, 4, 2002, 1640, 491, 16, 22, 1247, 37, 930, 258, 183, 931, 671, 401, 349, 1111, 1112, 1113, 1114, 1021, 8, 4, 553, 360, 16], [99, 203, 166, 1, 184, 3, 117, 3801, 148, 2, 52, 48, 3802, 22]]
label                                                   ham
sms       Thanks again for your reply today. When is ur ...
target                                                    0
Name: 4460, dtype: object
thanks again for your reply today when is ur visa coming in and r u still buying the gucci and bags my sister things are not easy uncle john also has his own bills so i really need to think abou

In [11]:
X_train_pad = pad_sequences(X_train_seq, maxlen=20, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=20, padding='post')
X_train_pad[:5]
X_train_pad.shape


(4458, 20)

In [12]:
laenge_pads = 20
anz_woerter = 7982

lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=anz_woerter+1, output_dim=20, input_length=laenge_pads))
lstm_model.add(LSTM(400))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.summary()

C:\Users\kunji\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [13]:
history = lstm_model.fit(X_train_pad, y_train, epochs=10, batch_size=64, 
                        validation_data=(X_test_pad, y_test))

Epoch 1/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 15s 135ms/step - accuracy: 0.8405 - loss: 0.3510 - val_accuracy: 0.9785 - val_loss: 0.0889
Epoch 2/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 9s 125ms/step - accuracy: 0.9853 - loss: 0.0593 - val_accuracy: 0.9847 - val_loss: 0.0609
Epoch 3/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 9s 125ms/step - accuracy: 0.9957 - loss: 0.0249 - val_accuracy: 0.9820 - val_loss: 0.0868
Epoch 4/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 9s 125ms/step - accuracy: 0.9990 - loss: 0.0034 - val_accuracy: 0.9767 - val_loss: 0.1374
Epoch 5/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 9s 124ms/step - accuracy: 0.9975 - loss: 0.0065 - val_accuracy: 0.9767 - val_loss: 0.1726
Epoch 6/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 9s 124ms/step - accuracy: 0.9987 - loss: 0.0093 - val_accuracy: 0.9785 - val_loss: 0.1372
Epoch 7/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 9s 125ms/step - accuracy: 0.9972 - loss: 0.0113 - val_accuracy: 0.9776 - val_loss: 0.1407
Epoch 8/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 9s 124ms/step - accuracy: 0.9952 - loss: 0.0176 - val_accuracy: 0

In [16]:
# Predict the class probabilities
predictions = lstm_model.predict(sms_pad)

# Get the predicted class index
predicted_class = np.argmax(predictions, axis=-1)

# Display the predicted class
print(predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 445ms/step
[0]


In [20]:
sms_test = ['I have been searching for the right words to thank you for this breather']
sms_seq = tokenizer.texts_to_sequences(sms_test)

sms_pad = pad_sequences(sms_seq, maxlen=20, padding='post')
tokenizer.index_word
sms_pad

# Updated method to get predictions
predicted_class = np.argmax(lstm_model.predict(sms_pad), axis=-1)
predicted_class


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step


array([0], dtype=int64)

In [34]:
# Import necessary libraries
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

# Load and preprocess dataset
df = pd.read_csv('spam.csv')
df['target'] = df['label'].map({'spam': 1, 'ham': 0})

# Separate training and test data
df_train = df.sample(frac=0.8, random_state=11)
df_test = df.drop(df_train.index)
print(df_train.shape, df_test.shape)

y_train = df_train['target'].values
y_test = df_test['target'].values

X_train = df_train['message'].values
X_test = df_test['message'].values

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
word_dict = tokenizer.index_word

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

print(X_train_seq[:5])
print(df_train.iloc[0, :])
for el in X_train_seq[0]:
    print(word_dict[el], end=' ')

# Padding sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=20, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=20, padding='post')
print(X_train_pad[:5])
print(X_train_pad.shape)

# Model parameters
laenge_pads = 20
anz_woerter = 7982

# Build LSTM model
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=anz_woerter + 1, output_dim=20, input_length=laenge_pads))
lstm_model.add(LSTM(400))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.summary()

# Train the model
history = lstm_model.fit(X_train_pad, y_train, epochs=10, batch_size=64, 
                         validation_data=(X_test_pad, y_test))

# Predict class for a test SMS
sms_test = ['I have been searching for the right words to thank you for this breather']
sms_seq = tokenizer.texts_to_sequences(sms_test)
sms_pad = pad_sequences(sms_seq, maxlen=20, padding='post')

# Updated method to get predictions
predicted_class = (lstm_model.predict(sms_pad) > 0.5).astype("int32")

# Display the predicted class
print(predicted_class)


(1346, 3) (337, 3)
[[2, 134, 84, 844, 179, 1344, 76, 30, 1, 22, 94, 33, 114, 1, 187], [159, 57, 415, 340, 21, 718, 12, 14, 3, 91, 630, 238], [94, 4, 78, 16, 5, 323, 1, 166, 167, 167], [94, 4, 66, 8, 560, 4, 631, 21, 1033], [719, 166, 217, 2074, 324]]
label                                                    ham
message    You tell what happen dont behave like this to ...
target                                                     0
Name: 1080, dtype: object
you tell what happen dont behave like this to me ok no need to say [[   2  134   84  844  179 1344   76   30    1   22   94   33  114    1
   187    0    0    0    0    0]
 [ 159   57  415  340   21  718   12   14    3   91  630  238    0    0
     0    0    0    0    0    0]
 [  94    4   78   16    5  323    1  166  167  167    0    0    0    0
     0    0    0    0    0    0]
 [  94    4   66    8  560    4  631   21 1033    0    0    0    0    0
     0    0    0    0    0    0]
 [ 719  166  217 2074  324    0    0    0    0    0  

C:\Users\kunji\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 151ms/step - accuracy: 0.5897 - loss: 0.5782 - val_accuracy: 0.9228 - val_loss: 0.2183
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - accuracy: 0.9365 - loss: 0.2729 - val_accuracy: 0.9466 - val_loss: 0.1759
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 118ms/step - accuracy: 0.9654 - loss: 0.1405 - val_accuracy: 0.9496 - val_loss: 0.1810
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 116ms/step - accuracy: 0.9883 - loss: 0.0381 - val_accuracy: 0.9496 - val_loss: 0.1530
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 124ms/step - accuracy: 0.9926 - loss: 0.0218 - val_accuracy: 0.9614 - val_loss: 0.1339
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 117ms/step - accuracy: 0.9957 - loss: 0.0106 - val_accuracy: 0.9228 - val_loss: 0.4723
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 117ms/step - accuracy: 0.9994 - loss: 0.0021 - val_accuracy: 0.9466 - val_loss: 0.2003
Epoch 8/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 119ms/step - accuracy: 0.9967 - loss: 0.0102 - val_accuracy: 0.

In [23]:
sms_test = ['Ringtone Club: Get the UK singles chart on your mobile each week and choose any top quality ringtone! This message is free of charge.']
sms_seq = tokenizer.texts_to_sequences(sms_test)
sms_pad = pad_sequences(sms_seq, maxlen=20, padding='post')

# Updated method to get predictions
predicted_class = (lstm_model.predict(sms_pad) > 0.5).astype("int32")

# Display the predicted class
print(predicted_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
[[1]]


In [24]:
# Save the trained model
lstm_model.save("spam_classifier_model.h5")  # Saves as an HDF5 file